In [ ]:
%matplotlib inline
from matplotlib import pylab as plt
from ipywidgets import interact
import ipywidgets as widgets
import matplotlib as mpl
import numpy as np
from geonotebook.wrappers import RasterDataCollection
import os

## Download the Data

The following cell is a bash one-liner for downloading monthly NBAR geotiffs from roughly Dec 2008 to Dec 2011. This data will download into a ```data/``` directory relative to the directory where this notebook is running. There are 36 files making up about **8Gb** of data. This may take some time to download so please be patient. You should recieve notification in the cell output for each downloaded file. 

For convenience here is an beautified version of the bash one-liner:

```bash
# Make the 'data' directory
if [ ! -d data ]; then 
  mkdir data; 
fi; 

# Recursively download the data via AWS CLI
if hash aws 2>/dev/null; then 
  aws s3 cp --recursive s3://golden-tile-geotiffs/ data; 
else 
  echo "AWS CLI script 'aws' is required"; 
fi
```

*Please note that the AWS command line script is required to download this data*

In [ ]:
!if [ ! -d data ]; then mkdir data; fi; if hash aws 2>/dev/null; then aws s3 cp --recursive s3://golden-tile-geotiffs/ data; else echo "AWS CLI script 'aws' is required"; fi

In [ ]:
DATA_DIR="data/"

In [ ]:
def sort_NBAR(a, b):
  am, ay = int(a.split(".")[2][-2:]), int(a.split(".")[3])
  bm, by = int(b.split(".")[2][-2:]), int(b.split(".")[3])

  if ay < by:
    return -1
  elif ay > by:
    return 1
  elif by == ay:
    if am < bm:
      return -1
    elif am > bm:
      return 1
    else:
      return 0
  

PATHS = [DATA_DIR + p for p in sorted(os.listdir(DATA_DIR), sort_NBAR)]

In [ ]:
PATHS

In [ ]:
rdc = RasterDataCollection(PATHS)

In [ ]:
M.add_layer(rdc[:,[1,2,3]], 'NBAR', opacity=0.8)

In [ ]:
M.layers["NBAR"].next()

In [ ]:
M.layers["NBAR"].prev()

In [ ]:
M.layers["NBAR"].seek(0)

In [ ]:
len(M.layers["NBAR"].data)

In [ ]:
def render_timeseries(idx=0):
  M.layers["NBAR"].seek(idx)
  
interact(render_timeseries, idx=(0, len(M.layers["NBAR"].data) - 1))

In [ ]:
M.layers["NBAR"].seek(1)

In [ ]:
M.remove_layer("NBAR")

In [ ]:
M.add_layer(rdc[:,[4]], 'NBAR_NDVI', opacity=0.8)

In [ ]:
M.remove_layer("NBAR_NDVI")

In [ ]:
def ndvi_colormap(numcolors=11, name='custom_div_cmap', 
                  mincol='blue', midcol='beige', maxcol='green'):
  return mpl.colors.LinearSegmentedColormap.from_list(
    name=name, colors=[mincol, midcol, maxcol], N=numcolors)

cmap = ndvi_colormap()


In [ ]:
colormap = [{"color": mpl.colors.rgb2hex(cmap(i)), "quantity": v }
 for i,v in zip(range(cmap.N),np.linspace(min(rdc[:,4].min), max(rdc[:,4].max), cmap.N))]

In [ ]:
colormap

In [ ]:
M.add_layer(rdc[:,4], 'NBAR_NDVI', colormap=colormap, opacity=0.8)

In [ ]:
M.layers['NBAR_NDVI'].next()

In [ ]:
M.layers